# Autoencoder asset pricing: A monte carlo simulation

In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
from torch.utils.data import DataLoader

from joblib import Parallel, delayed

from autoencoder import simulation as sim
from autoencoder.training import run
from autoencoder.networks import AE0, AE1, AE2, AE3
from autoencoder.utils import (
    split_dataset, torch_rolling_mean, AEDataset
)

In [3]:
# Constants
bs = 1
Ks = [3]
model_ids = [3]
models = {
    0: AE0,
    1: AE1,
    2: AE2,
    3: AE3
}

In [4]:
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'

print(f'Using {device} as device.')
device = torch.device(device)

Using mps as device.


In [5]:
plt.style.use('usual')

In [6]:
def run_model(i):

    model = models[model_id](sim.Pc, sim.Px, nb_factors).to(device)
    nb, lb, nr, lr, nm, lm,  c, f = sim.simulate()
    char_train, char_valid, char_tests = split_dataset(c)
    if is_linear:
        rets_train, rets_valid, rets_tests = split_dataset(lr)
        port_train, port_valid, port_tests = split_dataset(lm)
    
    else:
        rets_train, rets_valid, rets_tests = split_dataset(nr)
        port_train, port_valid, port_tests = split_dataset(nm)
    
    train = DataLoader(
        AEDataset(char_train, port_train, rets_train),
        batch_size=bs,
    )
    
    valid = DataLoader(
        AEDataset(char_valid, port_valid, rets_valid),
        batch_size=bs,
    )
    
    char_tests = torch.tensor(char_tests).to(device)
    rets_tests = torch.tensor(rets_tests).to(device)
    port_tests = torch.tensor(port_tests).to(device)
    # Train for each (model, number of factors) combinations
    
    
    lp, lr = 1e-4, 1e-3
    _, model, vl, tl, stop = run(model,
                                 device,
                                 train,
                                 valid,
                                 es_max_iter=20,
                                 epochs=200,
                                 lasso_param=lp,
                                 learning_rate=lr)
    
    # title = f'Sim{1}+AC{model_id}+K={nb_factors}'
    # plt.style.use('usual')
    # fig, ax = plt.subplots()
    # ax.set_ylabel(f'Training Error: AE{model_id}(K={nb_factors})')
    # ax.set_xlabel('Epochs')
    # ax.plot(tl, label='Training', color='black')
    # ax1 = plt.twinx()
    # ax1.set_ylabel('Validation Error')
    # ax1.plot(vl, label='Validation', color='red')
    # if stop > 0:
    #     ax1.axvline(stop, label='Early stopping', ls='--', color='blue')
    # fig.legend(ncol=3, loc='upper center')
    # fig.tight_layout()
    # if is_linear:
    #     fig.savefig(f'outputs/linear/{title}.pdf')
    # else:
    #     fig.savefig(f'outputs/non-linear/{title}.pdf')
    # ax.clear()
    # plt.close(fig)
    model.eval()
    with torch.no_grad():
    
        pred_tests = model(char_tests, port_tests)
        out = pred_tests
        rss = (pred_tests - rets_tests).pow(2).sum()
        tss = rets_tests.pow(2).sum()
        r2t = 1 - rss / tss
        
        factors = model.factors
        betas = model.loadings[1:]
        premias = torch_rolling_mean(factors, device)
        pred_tests = torch.squeeze(betas @ premias)
        out2 = pred_tests
        rss = (pred_tests - rets_tests[1:]).pow(2).sum()
        tss = rets_tests[1:].pow(2).sum()
        r2p = 1 - rss / tss

    return (model, r2t.item(), r2p.item())


This part can be put in a loop but the hyperparameters tuning must be performed first and the appropriate hyperparameters should be used.
Due to the time consuming part of hyperparameter tuning, I only heuristically tried to find the hyperparameters for the CA3 case with 3 factors. I also noticed that the model is quite sensitive to hyperparameters.

In [7]:
is_linear = False
model_id = 3
nb_factors = 3

In [8]:
results = Parallel(n_jobs=4)(delayed(run_model)(i) for i in range(100))

In [12]:
prdR2 = 0
totR2 = 0
for (r2t, r2p) in results:
     totR2 += r2t
     prdR2 += r2p


print(f'The results for the model AE{model_id} with K={nb_factors} are in the non-linear case')
print(f'Total R2 = {totR2:.2f}')
print(f'Predictive R2 = {prdR2:.2f}')

The results for the model AE3 with K=3 are in the non-linear case
Total R2 = 12.52
Predictive R2 = 1.82


In [13]:
is_linear = True

In [14]:
results = Parallel(n_jobs=4)(delayed(run_model)(i) for i in range(100))

In [15]:
prdR2 = 0
totR2 = 0
for (r2t, r2p) in results:
     totR2 += r2t
     prdR2 += r2p


print(f'The results for the model AE{model_id} with K={nb_factors} are in the linear case:')
print(f'Total R2 = {totR2:.2f}')
print(f'Predictive R2 = {prdR2:.2f}')

The results for the model AE3 with K=3 are in the linear case:
Total R2 = 26.84
Predictive R2 = 4.24


WE obtain similar results as in the paper.